In [1]:
import pandas as pd

df1 = pd.read_csv('Materials/2016_-_Cities_Emissions_Reduction_Targets_20240207.csv')
df2 = pd.read_csv('Materials/2016_-_Citywide_GHG_Emissions_20240207.csv')
df3 = pd.read_csv('Materials/2017_-_Cities_Community_Wide_Emissions.csv')
df4 = pd.read_csv('Materials/2017_-_Cities_Emissions_Reduction_Targets_20240207.csv')
df5 = pd.read_csv('Materials/2023_Cities_Climate_Risk_and_Vulnerability_Assessments_20240207.csv')

dataframes = [df1, df2, df3, df4, df5]

In [2]:
ac = 'Organisation Number'
ci = 'City'
org = 'Organisation'
co = 'Country'
c40 = 'C40 City'
em = 'Total emissions (metric tonnes CO2e)'
pop = 'City Population'
year = 'Reporting Year'

for df in dataframes:
    df.rename(columns={'Account No': ac, 'Organization Number': ac, 'Account number': ac,
                       'Organization Name': org, 'Organization': org,
                       'Country/Area': co,
                       'City Short Name': ci,
                       'Baseline emissions (metric tonnes CO2e)': em, 'Total City-wide Emissions (metric tonnes CO2e)': em,
                       'Population': pop, 'Current Population': pop,
                       'Reporting year': year,
                       'C40': c40}, inplace=True)

In [3]:
c40_to_bool = lambda x: False if pd.isnull(x) else (True if x == 'C40' else x)
for df in dataframes:
    df['C40 City'] = df['C40 City'].apply(c40_to_bool)

In [4]:
import re

def convert_to_geojson(value):
    if isinstance(value, str): 
        match = re.match(r'POINT \(([-+]?\d*\.\d+) ([-+]?\d*\.\d+)\)', value)
        if match:
            coordinates = [float(match.group(1)), float(match.group(2))]
            return {"type": "Point", "coordinates": coordinates}
        elif value.startswith('(') and value.endswith(')'):
            coordinates = [float(coord.strip()) for coord in value[1:-1].split(",")]
            return {"type": "Point", "coordinates": coordinates}
    else:
        return value

In [5]:
for df in dataframes:
    for col in df.columns:
        if 'Location' in col:
            df[col] = df[col].apply(convert_to_geojson)

In [6]:
for i,df in enumerate(dataframes):
    json_data = df.to_json(orient='records')

    filename = f'jsons/dataframe_{i + 1}.json'
    with open(filename, 'w') as f:
        f.write(json_data)